In [1]:
# importing required libraries
import pandas as pd
import json

```
{
        "textbook-paragraph": "{Error Correction}\n\nDetection of errors is importantn\n",
        "GPT-3-RLHF-Generations": {
            "question": " How can a",
            "answers": [
                " A digital to correct acurred.",
                " Error bit was in error and correct it accordingly.",
                " Hamming codes any two bit errors."
            ]
        }
    },
```

In [2]:
# load json from file (direct from Neha)
with open('RLHF_Keywords_Set1.json') as f:
  data = json.load(f)

# reformat
new_data = []
for el in data:
  newdict = {
    # 'textbook-paragraph': el['textbook-paragraph'],
    'question': el['GPT-3-RLHF-Generations']['question'],
    'answers': el['GPT-3-RLHF-Generations']['answers'],
  }
  new_data.append(newdict)
  new_data[0]

# to csv
df = pd.read_json(json.dumps(new_data))
df

,question,answers
0,What is a deterministic system and how does it...,[A deterministic system is one that gives the ...
1,How can a computer be seen as an interconnecte...,[A computer system is made up of many intercon...
2,What is the purpose of learning the C language...,[The purpose of learning the C language in com...
3,"How do computers differ from an ""electronic id...",[Computers are universal computational devices...
4,What is the difference between 0s and 1s in te...,"[In computer representation, 0s and 1s are kno..."
...,...,...
211,Can you explain how ASCII codes simplify the i...,[ASCII codes provide a standardized system for...
212,Why is the American Standard Code for Informat...,[The context does not mention the importance o...
213,How are ASCII codes stored and made available ...,"[When a key is typed on the keyboard, the comp..."
214,Why are most keys associated with more than on...,[Most keys on a keyboard are associated with m...


In [15]:
# Reformat for Scale.ai upalod
# [{"type": "text", "content": "#### Option 1\n-------\n**Answer 1**"}, {"type": "text", "content": "#### Option 2\n-------\n**Answer 2**"}, {"type": "text", "content": "#### Option 3\n-------\n**Answer 3**"}]
def json_gen(df):
  upload_entry = []
  meta_data = []
  count = 0
  for quest, ans in zip(df['question'], df['answers']):
    count += 1
    entry = [
        {
            "type": "text",
            "content": quest.split("?")[0] + "?\n" + "#### Option 1\n-------\n" + ans[0].strip()
        },
        {
            "type": "text",
            "content": "#### Option 2\n-------\n" + ans[1].strip()
        },
        {
            "type": "text",
            "content": "#### Option 3\n-------\n" + ans[2].strip()
        },
    ]
    # {"type": "text", "content": "#### Option 4\n-------\n" +  ans[3].strip()}]
    upload_entry.append(json.dumps(entry, indent=1))
    meta_data.append(json.dumps({'id': count}, indent=1))

  data = pd.DataFrame()
  data['attachments_json'] = upload_entry
  data['metadata'] = meta_data
  return data


# RUN IT
rlhf_data = json_gen(df)
# rlhf_data.to_csv('for_scale_rlhf_Keywords_Set1.csv', index=False)
# rlhf_data

,attachments_json,metadata
0,"[\n {\n ""type"": ""text"",\n ""content"": ""What i...","{\n ""id"": 1\n}"
1,"[\n {\n ""type"": ""text"",\n ""content"": ""How ca...","{\n ""id"": 2\n}"
2,"[\n {\n ""type"": ""text"",\n ""content"": ""What i...","{\n ""id"": 3\n}"
3,"[\n {\n ""type"": ""text"",\n ""content"": ""How do...","{\n ""id"": 4\n}"
4,"[\n {\n ""type"": ""text"",\n ""content"": ""What i...","{\n ""id"": 5\n}"
...,...,...
211,"[\n {\n ""type"": ""text"",\n ""content"": ""Can yo...","{\n ""id"": 212\n}"
212,"[\n {\n ""type"": ""text"",\n ""content"": ""Why is...","{\n ""id"": 213\n}"
213,"[\n {\n ""type"": ""text"",\n ""content"": ""How ar...","{\n ""id"": 214\n}"
214,"[\n {\n ""type"": ""text"",\n ""content"": ""Why ar...","{\n ""id"": 215\n}"


## Export to csv for upload to Scale.ai

In [23]:


rlhf_data_short = rlhf_data.iloc[0:100]
rlhf_data_short.to_csv('0_to_100_for_scale_rlhf_Keywords_Set1.csv', index=False)
rlhf_data_short

,attachments_json,metadata
0,"[\n {\n ""type"": ""text"",\n ""content"": ""What i...","{\n ""id"": 1\n}"
1,"[\n {\n ""type"": ""text"",\n ""content"": ""How ca...","{\n ""id"": 2\n}"
2,"[\n {\n ""type"": ""text"",\n ""content"": ""What i...","{\n ""id"": 3\n}"
3,"[\n {\n ""type"": ""text"",\n ""content"": ""How do...","{\n ""id"": 4\n}"
4,"[\n {\n ""type"": ""text"",\n ""content"": ""What i...","{\n ""id"": 5\n}"
...,...,...
95,"[\n {\n ""type"": ""text"",\n ""content"": ""Can yo...","{\n ""id"": 96\n}"
96,"[\n {\n ""type"": ""text"",\n ""content"": ""What d...","{\n ""id"": 97\n}"
97,"[\n {\n ""type"": ""text"",\n ""content"": ""Can yo...","{\n ""id"": 98\n}"
98,"[\n {\n ""type"": ""text"",\n ""content"": ""What i...","{\n ""id"": 99\n}"


# huggingface dataset to Scale.AI

In [7]:
# loaing test data
from datasets import load_dataset
test_dataset = load_dataset("kastan/EE_QA_for_RLHF")
test_dataset['train']

# test
test_dataframe = pd.DataFrame()
test_dataframe['textbook-paragraph'] = test_dataset['train']['textbook-paragraph']
test_dataframe['question'] = test_dataset['train']['question']
test_dataframe['answers'] = test_dataset['train']['answers']
test_dataframe.head()

/home/sbdesai2/utils/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration kastan--EE_QA_for_RLHF-971999be283e776c
Found cached dataset json (/home/sbdesai2/.cache/huggingface/datasets/kastan___json/kastan--EE_QA_for_RLHF-971999be283e776c/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 1/1 [00:00<00:00,  8.79it/s]


,textbook-paragraph,question,answers
0,"{Finite State Machine Design Examples, Part I}...",What is a finite state machine?\n,[\n\nA finite state machine is a mathematical ...
1,"{Finite State Machine Design Examples, Part I}...",What are the basic steps of designing a finit...,[ The basic steps of designing a finite state ...
2,"{Finite State Machine Design Examples, Part I}...",What is an example of a design that is more i...,[ A counter with a number of states that is no...
3,"{Finite State Machine Design Examples, Part I}...",What is the design process for a digital FSM?\n,[ The design process for a digital FSM is a pr...
4,"{Finite State Machine Design Examples, Part I}...",What is an example of a counter with a number ...,[ A counter with a number of states that is no...


In [8]:
# loading the rlhf data
f = open('/mnt/project/chatbotai/saish/data/RLHF_data_v2_1.json')
rlhf_dataset = json.load(f)

para, q, a = [],[],[]
for ele in rlhf_dataset:
    para.append(ele['textbook-paragraph'])
    q.append(ele['GPT-3-RLHF-Generations']['question'])
    a.append(ele['GPT-3-RLHF-Generations']['answers'])

rlhf_dataframe = pd.DataFrame()
rlhf_dataframe['textbook-paragraph'] = para
rlhf_dataframe['question'] = q
rlhf_dataframe['answers'] = a


rlhf_dataframe.head()


,textbook-paragraph,question,answers
0,"{Finite State Machine Design Examples, Part I}...",What is a finite state machine?\n,[ A finite state machine (FSM) is a model of c...
1,"{Finite State Machine Design Examples, Part I}...",What are the basic steps of designing a finit...,[ The basic steps of designing a finite state ...
2,"{Finite State Machine Design Examples, Part I}...",What is an example of a design that is more i...,[ A design that is more intuitive and easier f...
3,"{Finite State Machine Design Examples, Part I}...",What is the design process for a digital FSM?\n,[ The design process for a digital FSM typical...
4,"{Finite State Machine Design Examples, Part I}...",What is an example of a counter with a number ...,[ An example of a counter with a number of sta...


In [ ]:
test_data = json_gen(test_dataframe)
rlhf_data = json_gen(rlhf_dataframe)

In [14]:
rlhf_data.to_csv("ScaleRLHF_Keywords_Set1.csv", index = False)